In [78]:
from keras.models import load_model
import tensorflow as tf
import keras
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.metrics import classification_report

import numpy as np

In [71]:
data = pd.read_csv('../ds/Team/team.csv')

# Split the data into features (X) and labels (y)
X_new = data['Nhận xét'].astype(str).values
y_new = data['Cảm xúc'].values

data['Cảm xúc'] = data['Cảm xúc'].str.strip()
data.to_csv('team.csv', index=False)

label_encoder = LabelEncoder()

y_label =label_encoder.fit_transform(y_new)

X_train, X_val, y_train, y_val = train_test_split(X_new, y_label, test_size=0.2, random_state=132)
y_train_encoded = to_categorical(y_train)
y_val_encoded = to_categorical(y_val)


In [72]:


max_words = 5000
max_len = 1000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

X_train_seq = pad_sequences(X_train_seq, maxlen=max_len)
X_val_seq = pad_sequences(X_val_seq, maxlen=max_len)

In [73]:

# Load the pre-trained model
pretrained_model = tf.keras.models.load_model('../saved_model/sentiment_model.keras')

pretrained_model.summary()

for layer in pretrained_model.layers[:2]:  
    layer.trainable = False

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 120)         600000    
                                                                 
 conv1d (Conv1D)             (None, 996, 128)          76928     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 6)                 774       
                                                                 
Total params: 677,702
Trainable params: 677,702
Non-trainable params: 0
_________________________________________________________________


In [74]:

# Compile the model with the same optimizer and loss function
optimizer = keras.optimizers.Adam(learning_rate=0.0005)
pretrained_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])




In [75]:

# Train the model on new data
pretrained_model.fit(X_train_seq, y_train_encoded, batch_size=64, epochs=100, validation_data=(X_val_seq, y_val_encoded))

Epoch 1/100
161/161 [==============================] - 35s 211ms/step - loss: 2.7126 - accuracy: 0.2795 - val_loss: 2.1188 - val_accuracy: 0.3210
Epoch 2/100
161/161 [==============================] - 34s 214ms/step - loss: 1.9163 - accuracy: 0.3488 - val_loss: 1.6714 - val_accuracy: 0.3560
Epoch 3/100
161/161 [==============================] - 33s 208ms/step - loss: 1.5778 - accuracy: 0.4884 - val_loss: 1.4165 - val_accuracy: 0.5227
Epoch 4/100
161/161 [==============================] - 33s 205ms/step - loss: 1.3776 - accuracy: 0.5507 - val_loss: 1.2601 - val_accuracy: 0.5783
Epoch 5/100
161/161 [==============================] - 35s 220ms/step - loss: 1.2481 - accuracy: 0.5807 - val_loss: 1.1591 - val_accuracy: 0.6016
Epoch 6/100
161/161 [==============================] - 36s 224ms/step - loss: 1.1553 - accuracy: 0.5986 - val_loss: 1.0860 - val_accuracy: 0.6113
Epoch 7/100
161/161 [==============================] - 36s 225ms/step - loss: 1.0850 - accuracy: 0.6164 - val_loss: 1.0325 -

In [80]:
y_pred = pretrained_model.predict(X_val_seq)
# Convert numerical labels to original labels
y_pred_labels = np.argmax(y_pred, axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred_labels)

# Convert true labels to original labels
y_val_labels = label_encoder.inverse_transform(y_val)

# Generate classification report
report = classification_report(y_val_labels, y_pred_labels)
print(report)

 4/81 [>.............................] - ETA: 4s

81/81 [==============================] - 6s 72ms/step
                 precision    recall  f1-score   support

    bình thường       0.83      0.94      0.88       924
không liên quan       0.66      0.39      0.49       218
         rất tệ       0.33      0.16      0.21        32
        rất tốt       0.39      0.11      0.17        64
             tệ       0.81      0.72      0.76       567
            tốt       0.75      0.85      0.79       768

       accuracy                           0.79      2573
      macro avg       0.63      0.53      0.55      2573
   weighted avg       0.77      0.79      0.77      2573



In [81]:
pretrained_model.save('../saved_model/Transfer.keras')

